In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
import json
import numpy as np
import folium as g
import pandas as pd
import requests
import matplotlib as mpl

In [ ]:
mpl.rcParams['font.family'] = 'Noto Sans KR'
print(mpl.rcParams['font.family'])

In [ ]:
global_font = {'family':'NanumGothic','size':12,'weight':'bold'}

#폰트 설정
mpl.rc('font',**global_font)

## 지역화폐 사용 현황

- 데이터 가져오기

In [ ]:
local_g = pd.read_csv("datas/payuse.csv",encoding="CP949")
local_g

In [ ]:
local_g.기준년월 = local_g.기준년월.str.split('-').str[0]
local_g

In [ ]:
index = list(set(list(local_g.시군명)))

### 지역 별 카드 사용액 dataframe(use2019 - use2022)

In [ ]:
use2019 = local_g[local_g.기준년월 == "2019"]
use2020 = local_g[local_g.기준년월 == "2020"]
use2021 = local_g[local_g.기준년월 == "2021"]
use2022 = local_g[local_g.기준년월 == "2022"]

In [ ]:
pay= []
for i in index:    
    pay.append(use2019[use2019.시군명 == i]["월별카드사용액(천원)"].sum())
use2019 = pd.Series(data = pay, index = index)

pay= []
for i in index:    
    pay.append(use2020[use2020.시군명 == i]["월별카드사용액(천원)"].sum())
use2020 = pd.Series(data = pay, index = index)

pay= []
for i in index:    
    pay.append(use2021[use2021.시군명 == i]["월별카드사용액(천원)"].sum())
use2021 = pd.Series(data = pay, index = index)

pay= []
for i in index:    
    pay.append(use2022[use2022.시군명 == i]["월별카드사용액(천원)"].sum())
use2022 = pd.Series(data = pay, index = index)

In [ ]:
gg= "https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-municipalities-2018-geo.json"

response = requests.get(gg)
gg_json = response.json()

gg_list = [i['properties']['name'] for i in gg_json['features']]

In [ ]:
nindex = ['용인시', '수원시', '성남시', '안양시', '안산시', '고양시']

In [ ]:
ch = []
for i in gg_list:
    for j in nindex:
        if j in i:
            use2019[i]=use2019[j]
            use2020[i]=use2020[j]
            use2021[i]=use2021[j]
            use2022[i]=use2022[j]

In [ ]:
use2022.max()

In [ ]:
# threshold 넣을 값 
bins = [420000000*i for i in [0,0.01,0.10,0.20,0.30,0.40,0.60,0.80,1.00]]
bins[0]

In [ ]:
# 지역 별 카드사용액 지도
map = g.Map(location = [37.5502, 126.982], zoom_start = 8,tiles='cartodbpositron')
g.Choropleth(geo_data = gg,
                  data = use2019,
                  threshold_scale=bins,
                  fill_color='YlGn',
                  line_opacity=0.5,
                  fill_opacity=0.7,
                  key_on='feature.properties.name').add_to(map)
map

In [ ]:
map = g.Map(location = [37.5502, 126.982], zoom_start = 8,tiles='cartodbpositron')
g.Choropleth(geo_data = gg,
                  data = use2020,
                  threshold_scale=bins,
                  fill_color='YlGn',
                  line_opacity=0.5,
                  fill_opacity=0.7,
                  key_on='feature.properties.name').add_to(map)
map

In [ ]:
map = g.Map(location = [37.5502, 126.982], zoom_start = 8,tiles='cartodbpositron')
g.Choropleth(geo_data = gg,
                  data = use2021,
                  threshold_scale=bins,
                  fill_color='YlGn',
                  line_opacity=0.5,
                  fill_opacity=0.7,
                  key_on='feature.properties.name').add_to(map)
map

In [ ]:
map = g.Map(location = [37.5502, 126.982], zoom_start = 8,tiles='cartodbpositron')
g.Choropleth(geo_data = gg,
                  data = use2022,
                  threshold_scale=bins,
                  fill_color='YlGn',
                  line_opacity=0.5,
                  fill_opacity=0.7,
                  key_on='feature.properties.name').add_to(map)
map

# 지역 화폐 가맹점

- 데이터 가져오기

In [ ]:
pay_store=pd.read_csv("datas/lpay.csv", encoding = "CP949", low_memory = False, usecols = ["상호명" , "업종명(종목명)", "소재지도로명주소", "위도","경도","업종코드","폐업일자" ] )
pay_store

- 결측치 제거

In [ ]:
# 폐업 상점 삭제
pay_store = pay_store[pay_store.폐업일자.isna()]

# 위경도, 주소 결측치 제거
pay_store = pay_store[pay_store.위도.notnull()]
pay_store = pay_store[pay_store.소재지도로명주소.notnull()]

print(sum(pay_store.위도.isna()), sum(pay_store.폐업일자.notnull()))

In [ ]:
pay_store.drop(["폐업일자"], axis = 1, inplace =True)

In [ ]:
pay_store.drop(["상호명"], axis = 1, inplace =True)

In [ ]:
#기타 업종 제거
pay_store = pay_store[(pay_store["업종명(종목명)"]) != "기타"]
pay_store = pay_store[(pay_store["업종명(종목명)"]) != "기타1"]
pay_store = pay_store[(pay_store["업종명(종목명)"]) != "기타2"]
pay_store = pay_store[(pay_store["업종명(종목명)"]) != "기타3"]
pay_store = pay_store[(pay_store["업종명(종목명)"]) != "기타4"]
len(pay_store)

In [ ]:
pay_store["시"] = pay_store.소재지도로명주소.str.split(' ').str[1]

In [ ]:
#주소에서 구만 남기기

except_city = ['고양시', '수원시', '성남시', '용인시', '안양시', '안산시']

pay_store["시군구"] = pay_store.소재지도로명주소.str.split(' ').str[1]
except_city_idx = pay_store[pay_store.소재지도로명주소.str.split(' ').str[1].isin(except_city)].index

In [ ]:
pay_store.loc[except_city_idx, "시군구"] = pay_store.loc[except_city_idx, '소재지도로명주소'].str.split().str[1].astype("str") + pay_store.loc[except_city_idx, '소재지도로명주소'].str.split().str[2].astype("str")

In [ ]:
pay_store

- 업종 분류

In [ ]:
cat1 = ['일반자판기','기타음료식품','일반한식','서양음식','중국식', '스넥','갈비전문점','한정식','음식점업','일식회집','기타음료식품'] #요식업
cat2 = ['1급호텔', '2급호텔', '숙박업', '여관/기타숙박업', '콘도'] #숙박업
cat3 = ['쌍용S-OIL', '쌍용S-OIL가스충전소' , '주유소', 'E1가스충전소', 'GS가스충전소', 'GS주유소', 'LPG취급점', 'SK가스충전소', 'SK주유소','현대정유(오일뱅크)','현대정유가스충전소'] #주유소
cat4 = ['기타잡화', '내의판매', '정장', '신발', '스포츠의류', '아동의류', '내의판매', '맞춤복점', '정장(여성)', '캐주얼의류', '와이셔츠타이','단체복','기타의류','가방','액세서리',
        '귀금속','시계','안경'] #의류 잡화
cat5 = ['기타의료기관및기타의료기기','약국','의원', '치과의원', '한의원', '동물병원', '병원', '보건업', '한약방', '한방병원', '치과병원', '종합병원'] #병원
cat6 = ['제과점'] #카페/베이커리
cat7 = ['통신기기(무이자할부)', '통신기기', '일반(통신판매)','사무통신기기수리','통신판매업2'] #통신기기
cat8 = ['교육서비스', '교육서비스업', '기능학원', '예체능계학원', '외국어학원', '유아원', '보습학원', '기타교육기관', '유치원', '컴퓨터학원', '학원', '학습지교육', '독서실','유학원'] #교육서비스
cat9 = ['고속버스', '기타교통수단', '고속버스', '택시','교통수단'] #교통수단
cat10 = ['여객선','헬스클럽', '당구장', '수영장', '스포츠레져용품', '기타레져업소' , '골프용품전문점', '골프경기장', '골프연습장', '레져용품수리', '레져업소', '볼링장', '테니스장',
         '스포츠및여가관련서비스업', '이륜차판매','종합레져타운','스크린골프'] #레저/스포츠
cat11 = ['주점', '칵테일바','유흥주점'] #주점
cat12 = ['서적', '기타서적문구','전문서적','일반서적','화원', '노래방', '화랑','사우나', '티켓','문화취미기타','영화관','음반/영상물','수족관','회원제형태업소','관광여행'] #문화/취미
cat13 = ['기계공구','애완동물', '상품권', '과학기자재', '냉열기기', '기타광학제품', '카메라','대형할인점', '농축수산가공품','기타전기제품', '사무용OA기기', '기타농업관련','농기계', '조명기구', '보일러펌프샷시','일반가구', '목재석재철물','가전제품',
         '농축수산품','정육점','주방용식기','주방용구','카페트,커튼,천막,지물','악기점','농,축협직영매장','농협하나로클럽','소매업','화방표구점','개인서비스업', '건강식품',
         '골동품점', '구내매점', '기념품점', '기타건강식품', '기타사무용품', '기타용품', '문구용품', '문화취미기타', '미용재료', '기타직물', '비료','사료','종자', '성인용품점',
         '완구점', '의료용품', '주류판매점', '페인트', '홍삼제품'] #도/소매업
cat14 = ['슈퍼마켓', '편의점'] #편의점
cat15 = ['중고자동차판매','렌터카','중장비수리','주차장','세차장','카인테리어','가타자동차서비스', '견인서비스', '수입자동차', '자동차부품', '자동차서비스', '자동차시트타이어', '자동차정비',
         '중고차판매', '국산신차직영부품정비업소','국산신차판매',] #자동차 관련업
cat16 = ['화장품', '피부미용실', '미용원','이용원'] #뷰티
cat17 = ['여성청소년생필품','비영리/대상', '비영리/비대상'] #비영리
cat18 = ['기타용역서비스','용역서비스',
       '기타유통업', '사진관', '기타건축자재', '기타대인서비스', '인테리어전문', '세탁소',
       '기타가구', '기타운송', '기타전문점', '건축용요업제품', '침구수예점', '소프트웨어',
       '위탁급식업', '가정용품수리', '유류판매', '레저/서비스', '부동산중개임대', '출판및인쇄물',
       '법률회계서비스(개인)', '화물운송', '민예공예품', '종합용역', '중고자동차판매', '제화', '컴퓨터',
       '유리', '산후조리원', '인삼제품', '안마시술소', '비료,사료,종자', '옷감직물', '윤활유전문판매',
       '미곡상', '기타수리서비스', '정보서비스', '혼수전문점', '가례서비스업', '인터넷Mall',
       '혼례서비스', '기타연료', '법률회계서비스(법인)', '신변잡화수리', '조세서비스', '구내매점(국가기관등)',
       '정수기', '보관및창고업', '정기간행물', '피아노대리점', '철제가구', '기타주방용구', '복지매장',
       '자체물품대금', '상품권전문판매', '기타보험', '제약회사', '전자(상우회)', '양품점',
       '조산원', '조세서비스(특약)', '연쇄점', '제조업', '교육용테이프판매', '부동산분양', 'CATV'] # 서비스/기타

In [ ]:
pay_store.replace(cat1,'요식업', inplace =True)
pay_store.replace(cat2,'숙박업', inplace =True)
pay_store.replace(cat3,'주유소', inplace =True)
pay_store.replace(cat4,'의류/잡화', inplace =True)
pay_store.replace(cat5,'병원', inplace =True)
pay_store.replace(cat6,'카페/베이커리', inplace =True)
pay_store.replace(cat7,'통신기기', inplace =True)
pay_store.replace(cat8,'교육', inplace =True)
pay_store.replace(cat9,'교통수단', inplace =True)
pay_store.replace(cat10,'레저/스포츠', inplace =True)
pay_store.replace(cat11,'주점', inplace =True)
pay_store.replace(cat12,'문화/취미', inplace =True)
pay_store.replace(cat13,'도/소매업', inplace =True)
pay_store.replace(cat14,'편의점', inplace =True)
pay_store.replace(cat15,'자동차 관련업', inplace =True)
pay_store.replace(cat16,'뷰티', inplace =True)
pay_store.replace(cat17,'비영리', inplace =True)
pay_store.replace(cat18,'서비스/기타', inplace =True)

In [ ]:
pay_store["업종명(종목명)"].unique()

In [ ]:
pay_store

- 가맹점 데이터 시각화

In [ ]:
plt.figure(figsize = (5,10))
fig = sns.barplot(x = pay_store.시.value_counts(), y = pay_store.시.value_counts().index)
plt.xlabel(None)
plt.ylabel(None)
plt.title('시 별 가맹점 수 합계')
plt.show()

In [ ]:
plt.figure(figsize = (5,10))
sns.barplot(x = pay_store["업종명(종목명)"].value_counts(), y =  pay_store["업종명(종목명)"].value_counts().index)
plt.xlabel(None)
plt.ylabel(None)
plt.title('업종별 가맹점 수 합계')
plt.show()

In [ ]:
# 업종 비율 트리맵
! pip install squarify
import squarify
plt.rcParams['font.size'] = 20
fig, ax = plt.subplots(1, figsize = (20,10))
squarify.plot(pay_store["업종명(종목명)"].value_counts(),label=pay_store["업종명(종목명)"].value_counts().index)
plt.axis('off')
plt.show()

- 지도 시각화

In [ ]:
#시 별 가맹점 수 데이터 프레임 생성
gg_city = pd.Series(data = pay_store.시군구.value_counts(),
                  index = pay_store.시군구.value_counts().index)


In [ ]:
gg_city

In [ ]:
# 한국 좌표 geojson
geojson = "https://raw.githubusercontent.com/southkorea/southkorea-maps/master/kostat/2018/json/skorea-municipalities-2018-geo.json"

In [ ]:
text_data = urllib.request.urlopen(geojson).read().decode('utf-8')
gg = json.loads(text_data)

In [ ]:
# 고양시, 수원시, 성남시, 용인시, 안양시, 안산시
gg_city

In [ ]:
city_names = [feature['properties']['name'] for feature in gg['features']]
city_names

In [ ]:
gg_city

In [ ]:
bins = [np.percentile(gg_city.values, i) for i in range(0, 101, 25)]
bins

In [ ]:
#가맹점 분포 지도

map = g.Map(location = [37.5502, 126.982], zoom_start = 8,tiles='cartodbpositron')
g.Choropleth(geo_data = geojson,
                  data = gg_city,
                  # columns= [gg_city.index, gg_city],
                  threshold_scale=bins,
                  fill_color='YlGn',
                  line_opacity=0.5,
                  fill_opacity=0.7,
                  key_on='feature.properties.name').add_to(map)
map

### 연령별/성별 지역화폐 이용 현황

In [ ]:
# 연령별, 성별 이용현환 csv 파일 불러오기
sex = pd.read_csv('datas/payage.csv',index_col = False, low_memory=False)

In [ ]:
sex = sex[(sex['성별'] == '남') | (sex['성별'] == '여')]  # 불러온 파일의 column중 성별에서 남,여만 나오게 추출
sex["연령대"] = sex['연령대'].str[:2] # 연령대에서 두 글자까지의 개수만 출력 
sex=sex.iloc[:,0:6]   # 성별이용현황 성별: nan값 뺀 거
sex2021 = sex[sex.기준연도==2021]
sex21=sex2021.set_index('기준연도')

In [ ]:
se = sex21[(sex21['연령대'] == '10')|(sex21['연령대'] == '20')|(sex21['연령대'] == '30')|(sex21['연령대'] == '40')|(sex21['연령대'] == '50')|(sex21['연령대'] == '60')|(sex21['연령대'] == '70')]
se   # 연령대에서 10~70대까지의 값만 가질수있게 함

In [ ]:
# 불러온 csv 파일에 겹치는 데이터가 많아서 type을 바꿔주고 drop을 사용하여 중복된 값 삭제
se.성별 = se.성별.astype('category')
se.연령대 = se.연령대.astype('int')

In [ ]:
se.info()

In [ ]:
se.drop_duplicates(inplace=True)

In [ ]:
man = se[se['성별'] == '남']  # 남자 데이터만 나오게 한다.
man

In [ ]:
man_10_1 = man[man['연령대'] == 10]
man_20_1 = man[man['연령대'] == 20]
man_30_1 = man[man['연령대'] == 30]
man_40_1 = man[man['연령대'] == 40]
man_50_1 = man[man['연령대'] == 50]
man_60_1 = man[man['연령대'] == 60]
man_70_1 = man[man['연령대'] == 70]

In [ ]:
# 결제금액의 합을 구하고 경기도 연령별 남자 인구를 나누면
# 경기도 연령별 연 평균 결제금액이 나온다.
a = round((man_10_1['결제금액'].sum()/698771),0)
b = round((man_20_1['결제금액'].sum()/1273058),0)
c = round((man_30_1['결제금액'].sum()/1301183),0)
d = round((man_40_1['결제금액'].sum()/1368463),0)
e = round((man_50_1['결제금액'].sum()/1229784),0)
f = round((man_60_1['결제금액'].sum()/827358),0)
g = round((man_70_1['결제금액'].sum()/463035),0)

In [ ]:
woman = se[se['성별'] == '여'] # 여자 데이터만 출력한다.

In [ ]:
woman_10_1 = woman[woman['연령대'] == 10] #10대
woman_20_1 = woman[woman['연령대'] == 20] #20대
woman_30_1 = woman[woman['연령대'] == 30] #30대
woman_40_1 = woman[woman['연령대'] == 40] #40대
woman_50_1 = woman[woman['연령대'] == 50] #50대
woman_60_1 = woman[woman['연령대'] == 60] #60대
woman_70_1 = woman[woman['연령대'] == 70] #70대

In [ ]:
# 결제금액의 합을 구하고 경기도 연령별 여성 인구를 나눈다
q = round((woman_10_1['결제금액'].sum()/696715),0) 
w = round((woman_20_1['결제금액'].sum()/1197362),0)
e = round((woman_30_1['결제금액'].sum()/1215053),0)
r = round((woman_40_1['결제금액'].sum()/1288006),0)
t = round((woman_50_1['결제금액'].sum()/1227040),0)
n = round((woman_60_1['결제금액'].sum()/843535),0)
u = round((woman_70_1['결제금액'].sum()/539107),0)

In [ ]:
# 남자 연령별 1인 연 평균 사용 비율 데이터프레임으로 만든다
x = ['10대','20대','30대','40대','50대','60대','70대']
y = [a,b,c,d,e,f,g]

df = pd.DataFrame({'연령':x,'화폐':y}, columns=['연령','화폐'])
df

In [ ]:
# 여자 연령별 1인 연 평균 사용 비율 데이터프레임으로 만든다
x = ['10대','20대','30대','40대','50대','60대','70대']
y = [q,w,e,r,t,n,u] 

wy = pd.DataFrame({'연령':x,'화폐':y}, columns=['연령','화폐'])
wy

In [ ]:
df['성별'] = '남'  # 성별 column 추가
df

In [ ]:
wy['성별'] = '여' 
wy

In [ ]:
# 남, 여 데이터프레임 합침
result = pd.concat([df, wy], axis=0) 

In [ ]:
# 인덱스 값 0부터 순차적으로 진행되게 한다.
result.reset_index(inplace=True, drop=True)
result

In [ ]:
ax = sns.lineplot(data=result, x ='연령', y='화폐', hue= '성별')
plt.title('경기도 지역화폐 성별 1인 연평균 결제량')
current_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.0f}'.format(x) for x in current_values])

for patch in ax.patches:
    # 2개의 patch 값을 가짐
    ax.text(x = patch.get_x(),# + 100,
            y = patch.get_height(),
            s = f'{(patch.get_height()):1.0f}',
            ha = 'left')
plt.rc('font', size=11)
plt.savefig('성별 이용량.png',bbox_inches='tight')

# 지역및 연도별 카드 사용액

In [ ]:
local_g = pd.read_csv('datas/지역화폐발행및이용현황.csv', encoding='CP949', low_memory=False)
local_g = local_g.drop(['월별모바일상품권이용등록자수(명)', '월별모바일충전액(천원)', '월별모바일사용액(천원)', '월별지류상품권판매액(천원)', '월별지류상품권회수액(천원)'], axis=1)

local_g['기준년월'] = local_g['기준년월'].str.split('-').str[0] 

local_g1 = local_g.groupby(['시군명', '기준년월']).sum()
local_g1 = local_g1.drop(['월별카드발행수량(건)', '월별카드충전액(천원)'], axis=1)

# 도시별 년도별 총 사용량으로 나눔
temp3 = local_g1.reset_index()
pivot_temp = pd.pivot_table(temp3,
               index = "시군명",
               columns = "기준년월",
               values = "월별카드사용액(천원)")
pivot_temp

temp4 = pivot_temp.reset_index()
temp4.drop([7, 13], inplace=True)
temp4

ax = temp4.plot(x='시군명', kind='bar', figsize=(30, 20), rot=0)

# x, y축 레이블, 그래프 제목 설정

ax.set_xlabel('지역')
ax.set_ylabel('카드사용액')
ax.set_title('지역 및 연도별 카드 사용액')

plt.xticks(rotation=90)



# 범례 추가
ax.legend(title='Year')

plt.show()

In [ ]:
local_g = pd.read_csv('datas/지역화폐발행및이용현황.csv', encoding='CP949', low_memory=False)

In [ ]:
local_22 = local_g[local_g['기준년월'] > '2022']  ## 22년도 경기도 자료
local_21 = local_g[local_g['기준년월'] > '2021'] 
local_21 = local_21[local_21['기준년월'] < '2022']  ## 21년도 경기도 자료
local_20 = local_g[local_g['기준년월'] > '2020']
local_20 = local_20[local_20['기준년월'] < '2021']  ## 20년도 경기도 자료
local_19 = local_g[local_g['기준년월'] > '2019']
local_19 = local_19[local_19['기준년월'] < '2020']  ## 19년도 경기도 자료

total_19 = local_19[['월별카드발행수량(건)', '월별카드충전액(천원)', '월별카드사용액(천원)']].sum()
total_20 = local_20[['월별카드발행수량(건)', '월별카드충전액(천원)', '월별카드사용액(천원)']].sum()
total_21 = local_21[['월별카드발행수량(건)', '월별카드충전액(천원)', '월별카드사용액(천원)']].sum()
total_22 = local_22[['월별카드발행수량(건)', '월별카드충전액(천원)', '월별카드사용액(천원)']].sum()

total_19
print(total_21)
total_21

In [ ]:
# 지역화폐발행및이용현황.csv 파일에서 필요없는 컬럼 삭제하고 카드 자료만 남김
local_g = local_g.drop(['월별모바일상품권이용등록자수(명)', '월별모바일충전액(천원)', '월별모바일사용액(천원)', '월별지류상품권판매액(천원)', '월별지류상품권회수액(천원)'], axis=1)

In [ ]:
# 년도별 시군별 카드 사용액으로 나눔
local_g['기준년월'] = local_g['기준년월'].str.split('-').str[0] 

local_g1 = local_g.groupby(['시군명', '기준년월']).sum()
local_g1 = local_g1.drop(['월별카드발행수량(건)', '월별카드충전액(천원)'], axis=1)
local_g1

In [ ]:
#  년도별 시군별 총 사용액으로 나눔
temp3 = local_g1.reset_index()
pivot_temp = pd.pivot_table(temp3,
               index = "시군명",
               columns = "기준년월",
               values = "월별카드사용액(천원)")
pivot_temp

temp4 = pivot_temp.reset_index()
temp4.drop([7, 13], inplace=True)  # 자료가 없는 시흥과 김포 삭제
temp4

In [ ]:
# 19년도 부터 22년도까지의 시, 및 군별 사용 총액  김포시와 시흥시는 값이 없어서 삭제함
temp1 = local_g.groupby('시군명').sum()
temp1 = temp1.rename({'월별카드발행수량(건)': '카드발행수량',
              '월별카드충전액(천원)': '카드충전액',
              '월별카드사용액(천원)': '카드사용액'
             }, axis=1).reset_index().drop(['카드충전액','카드발행수량'], axis=1)
temp1['카드사용액(백만원)'] = temp1['카드사용액'] / 1000
temp1.drop('카드사용액', axis=1, inplace=True)
temp1.sort_values(by='카드사용액(백만원)', ascending=False, inplace = True)
temp1.drop([13, 7], inplace=True)

temp1

# 경기도 시군별 카드 총사용액 그래프

In [ ]:
# 경기도 시군별 카드 총 사용액
plt.figure(figsize=(50,30))  
plt.rc('font', size=20)
plt.rc('axes', labelsize=40)   
plt.rc('xtick', labelsize=40)   
plt.rc('ytick', labelsize=40)  
plt.xticks(rotation = 0)
sns.barplot(data=temp1, x="카드사용액(백만원)", y="시군명") 
plt.title('경기도 시군별 카드 총사용액(19 - 22년)', size=70)
plt.xlabel('카드 사용액')

plt.show()


In [ ]:
# 경기도 년도별 사용량 비교를 위해 자료를 다시 불러옴.
local_g = pd.read_csv('datas/지역화폐발행및이용현황.csv', encoding='CP949', low_memory=False)

In [ ]:
local_22 = local_g[local_g['기준년월'] > '2022']  ## 22년도 경기도 자료
local_21 = local_g[local_g['기준년월'] > '2021'] 
local_21 = local_21[local_21['기준년월'] < '2022']  ## 21년도 경기도 자료
local_20 = local_g[local_g['기준년월'] > '2020']
local_20 = local_20[local_20['기준년월'] < '2021']  ## 20년도 경기도 자료
local_19 = local_g[local_g['기준년월'] > '2019']
local_19 = local_19[local_19['기준년월'] < '2020']  ## 19년도 경기도 자료

total_19 = local_19[['월별카드발행수량(건)', '월별카드충전액(천원)', '월별카드사용액(천원)']].sum()
total_20 = local_20[['월별카드발행수량(건)', '월별카드충전액(천원)', '월별카드사용액(천원)']].sum()
total_21 = local_21[['월별카드발행수량(건)', '월별카드충전액(천원)', '월별카드사용액(천원)']].sum()
total_22 = local_22[['월별카드발행수량(건)', '월별카드충전액(천원)', '월별카드사용액(천원)']].sum()

total_19
print(total_21)
total_21

In [ ]:
# 도시별 년도별 총 사용량으로 나눔
temp3 = local_g1.reset_index()
pivot_temp = pd.pivot_table(temp3,
               index = "시군명",
               columns = "기준년월",
               values = "월별카드사용액(천원)")
pivot_temp

temp4 = pivot_temp.reset_index()
temp4.drop([7, 13], inplace=True)
temp4

In [ ]:
# 연도별 경기도 자료를 데이터프레임화 시키기

a = pd.DataFrame(total_19)
b = pd.DataFrame(total_20)
c = pd.DataFrame(total_21)
d = pd.DataFrame(total_22)

total = pd.DataFrame([total_19, total_20, total_21, total_22])
total = total.rename({0: '2019년',
              1: '2020년',
              2: '2021년',
              3: '2022년'
             })
total = total.rename({'월별카드발행수량(건)' : '카드발행수량(건)',
              '월별카드충전액(천원)' : '카드충전액(천원)',
              '월별카드사용액(천원)' : '카드사용액(천원)'} , axis=1)

total[['카드충전액(백만원)', '카드사용액(백만원)']] = total[['카드충전액(천원)', '카드사용액(천원)']] / 1000  # 단위가 크기때문에 줄여줌

total.drop(['카드충전액(천원)', '카드사용액(천원)'], axis=1, inplace=True)
total.reset_index(inplace=True)
total.rename({'index': '년도'}, axis=1, inplace=True)

# 경기도 연도별 카드발행 수량 및 충전액/사용액 그래프

In [ ]:
# 그래프 그리기
fig, ax1 = plt.subplots(figsize=(20, 10))

n = len(total['년도'].unique())
index = np.arange(n)

bar_width = 0.3
ax1.bar(index - bar_width/2, total['카드충전액(백만원)'], width=bar_width, color='green', label='카드충전액(백만원)')
ax1.bar(index + bar_width/2, total['카드사용액(백만원)'], width=bar_width, color='blue', label='카드사용액(백만원)')
ax1.legend(loc=2)
ax1.set_ylabel('카드충전액 / 카드사용액', fontsize=25)
ax2 = ax1.twinx()
ax2.set_xlabel('년도', fontsize=10)
ax2.set_ylabel('카드발행수량', fontsize=25)
ax2.plot(total['년도'], total['카드발행수량(건)'], marker='s', color='red')
ax2.legend(['카드발행수량(건)'], loc=6)


plt.title('경기도 연도별 카드발행 수량 및 충전액/사용액', fontsize=35)
plt.grid(visible = False)

plt.show()

In [ ]:
# 인구 및 지역화폐 이용 내역의 차이를 보여주기 위해 주민등록 인구 집계 현황을 불러옴
population = pd.read_csv('datas/주민등록인구집계현황.csv', encoding='CP949', low_memory=False)
population['행정구역명'] = population['행정구역명'].str.strip()


# 필요없는 컬럼 삭제
population = population.drop(['월', '0~9세', '10~19세', '20~29세', '30~39세',
       '40~49세', '50~59세', '60~69세', '70~79세', '80~89세', '90~99세', '100세 이상',
       '총 인구수 (남)', '0~9세 (남)', '10~19세 (남)', '20~29세 (남)', '30~39세 (남)',
       '40~49세 (남)', '50~59세 (남)', '60~69세 (남)', '70~79세 (남)', '80~89세 (남)',
       '90~99세 (남)', '100세 이상 (남)', '총 인구수 (여)', '0~9세 (여)', '10~19세 (여)',
       '20~29세 (여)', '30~39세 (여)', '40~49세 (여)', '50~59세 (여)', '60~69세 (여)',
       '70~79세 (여)', '80~89세 (여)', '90~99세 (여)', '100세 이상 (여)'], axis=1)

# 인구에서 시군으로만 나오게 
population = population[population['행정구역구분명'] == '시군']

# 행정구역 보기 쉽게 
population['행정구역명_2'] = population['행정구역명'].str.split(' ').str[1]  # 행정구역 보기 쉽게
population = population.drop(['행정구역구분명', '행정구역명'], axis=1)
population.reset_index(drop = True)

# 인구 연도 19년도에서 22년 까지만 비교를 하기 위해 나머지 년도의 자료는 삭제
population = population[population['연도'] >= 2019]
population = population[population['연도'] <= 2022]


In [ ]:
population.astype({"총 인구수": "int"})

In [ ]:
# 인구 및 지역화폐 분석을 위해 다시 불러옴
temp1 = local_g.groupby('시군명').sum()
temp1 = temp1.rename({'월별카드발행수량(건)': '카드발행수량',
              '월별카드충전액(천원)': '카드충전액',
              '월별카드사용액(천원)': '카드사용액'
             }, axis=1).reset_index().drop(['카드충전액','카드발행수량', '월별모바일상품권이용등록자수(명)', '월별모바일충전액(천원)', '월별모바일사용액(천원)', '월별지류상품권판매액(천원)','월별지류상품권회수액(천원)'], axis=1)
temp1['카드사용액(백만원)'] = temp1['카드사용액'] / 1000
temp1.drop('카드사용액', axis=1, inplace=True)
temp1.sort_values(by='카드사용액(백만원)', ascending=False, inplace = True)
temp1.drop([13, 7], inplace=True)

use = temp1
use = use.sort_values('시군명')  

use

In [ ]:
pivot_temp1 = pd.pivot_table(population,
               index = "행정구역명_2",
               columns = "연도",
               values = "총 인구수",
               aggfunc = 'mean' )

# 필요 없는 행정구역 삭제
a = pivot_temp1.reset_index()
a.drop([17, 30, 33], axis=0, inplace=True)
a.set_index('행정구역명_2')

# 경기도 시군별 19년도부터 21년도까지의 평균 인구를 구하기
a['인구평균'] = a.mean(axis='columns')

pop_means = a.drop([2019, 2020, 2021, 2022], axis=1)  # 행정구역 별 인구 평균

pop_means['인구비율'] = pop_means["인구평균"]/pop_means["인구평균"].sum()*100  # 구한 평균을 통해 인구 비율을 구함

pop_means = pop_means.drop([7, 13], axis=0) # 자료가 없었던 김포시와 시흥시 삭제

pop_means = pop_means.set_index('행정구역명_2')

In [ ]:
use['temp'] = use['카드사용액(백만원)'] / use['카드사용액(백만원)'].sum()*100  #카드 사용액의 비율을 구하기 위한 임시 파일
use = use.set_index('시군명')  # 시군 정렬
pop_means['총사용액'] = use['카드사용액(백만원)'] 
pop_means['사용비율'] = pop_means['총사용액']/pop_means["총사용액"].sum()*100
pop_means=pop_means.reset_index()


In [ ]:
import matplotlib as mpl
# Add every font at the specified location
font_dir = ['./fonts']
for font in mpl.font_manager.findSystemFonts(fontpaths=font_dir):
    print(mpl.font_manager.FontProperties(fname=font).get_name())
    mpl.font_manager.fontManager.addfont(font)

In [ ]:
mpl.rcParams['font.family'] = 'Noto Sans KR'
print(mpl.rcParams['font.family'])

In [ ]:
global_font = {'family':'NanumGothic','size':12,'weight':'bold'}

#폰트 설정
mpl.rc('font',**global_font)

# 인구 및 지역화폐 결제금액 비율

In [ ]:
#3.인구비율과 지역화폐 결제금액 비율을 동시에 시각화

pop_df = pop_means[["행정구역명_2", "인구비율","사용비율"]]  
pop_df = pop_df.set_index("행정구역명_2") 

# 시군구별 인구 및 지역화폐 결재금액 시각화
ax=pop_df.sort_values(by='인구비율', ascending=False).plot(kind='bar', figsize=(12, 8), legend=True, fontsize=12, color=['#E866BA','#98B3FF']) 
ax.set_xlabel('지역명', fontsize=12) 
ax.set_ylabel('비율', fontsize=12) 
plt.title("인구 및 지역화폐 결제금액 비율", fontsize=15)
ax.legend(['거주인구비율', '결제금액비율'], fontsize=12) 
plt.show()

# 인구 및 지역화폐 결제금액 그래프

In [ ]:
# 인구평균과 총사용액 비교

pop_df2 = pop_means[["행정구역명_2", "인구평균","총사용액"]]  

pop_df2 = pop_df2.set_index('행정구역명_2')

# 인구 및 지역화폐 결제 금액에 대한 표
ax=pop_df2.sort_values(by='인구평균', ascending=False).plot(kind='bar', figsize=(12, 8), legend=True, fontsize=12, color=['#E866BA','#98B3FF']) 
ax.set_xlabel('인구평균', fontsize=12) 
ax.set_ylabel('인구수 및 결재금액', fontsize=12) 
plt.title("인구 및 지역화폐 결제금액", fontsize=15)
ax.legend(['총거주인구수', '결제금액'], fontsize=12) 


In [ ]:
# 가구수에 따른 변화를 확인 하기 위한 작업 파일
family = pd.read_csv('datas/세대원수별세대수집계현황.csv', encoding='CP949', low_memory=False)

# 19년도 부터 21년까지의 자료만 사용
family = family[family['연도'] >= 2019]

# 시군 데이터만 사용
family = family[family['행정구역구분명'] == '시군'] 
family['행정구역명'] = family['행정구역명'].str.split(' ').str[1]

#필요 없는 데이터 삭제
family.drop(['행정구역구분명'], axis=1, inplace=True)

# 인덱스 수정
family.set_index('행정구역명', inplace=True)

# 3인 이상의 세대수는 하나로 통합
family['3인 이상 세대수'] = family['3인 세대수'] + family['4인 세대수'] + family['5인 세대수'] + family['6인 세대수'] + family['7인 세대수'] + family['8인 세대수'] + family['9인 세대수'] + family['10인이상 세대수'] 

# 필요 없는 데이터 삭제
family = family.drop(['3인 세대수', '4인 세대수', '5인 세대수', '6인 세대수', '7인 세대수', '8인 세대수', '9인 세대수', '10인이상 세대수'], axis=1)
family = family[family['연도'] < 2023]


family = family.groupby(['행정구역명','연도']).mean().drop(['월'], axis=1)

family = family.groupby('행정구역명').mean()
family = family.reset_index().drop([6, 13])
family.reset_index()

In [ ]:
family4 = family.loc[[12, 11, 30, 4]]
family4.sort_values('행정구역명') 

In [ ]:
family5 = family4.set_index('행정구역명').sort_index()
family5

# 가구별 비율 나타내기

In [ ]:
family5["1인가구 비율"] = family5["1인 세대수"] / family5["전체 세대수"]*100
family5["2인가구 비율"] = family5["2인 세대수"] / family5["전체 세대수"]*100
family5["3인이상 비율"] = family5["3인 이상 세대수"] / family5["전체 세대수"]*100

In [ ]:
family5 = family5.astype({"1인 세대수": "int",
              "2인 세대수": "int",
              "3인 이상 세대수": "int"})

In [ ]:
family6 = family5.drop(['전체 세대수', '1인 세대수', '2인 세대수', '3인 이상 세대수'], axis=1)
family6

In [ ]:
# 수원시 가구비율
labels = ['1인가구 비율', '2인가구 비율', '3인이상 가구비율']
sizes = [38.1, 20.2, 41.7]
colors = ['gold', 'whitesmoke', 'lightgray']
explode = (0, 0, 0.1)
plt.title('수원시 가구비율', size=30)
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=90)
plt.axis('equal')  # 원의 형태를 유지 
plt.rc('figure', titlesize=5)
plt.show()

In [ ]:
# 광주시 가구비율
labels = ['1인가구 비율', '2인가구 비율', '3인이상 가구비율']
sizes = [35.2, 24.1, 40.6]
colors = ['#ff9999', '#ffc000', '#8fd9b6']
explode = (0, 0, 0.1)
plt.title('광주시 가구비율', size=30)
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=90)
plt.axis('equal')  # 원의 형태를 유지 
plt.show()

In [ ]:
# 성남시 가구비율
labels = ['1인가구 비율', '2인가구 비율', '3인이상 가구비율']
sizes = [37.7, 21.7, 40.6]
colors = ['#ffc000', '#8fd9b6', '#d395d0']
explode = (0, 0, 0.1)
plt.title('성남시 가구비율', size=30)
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=90)
plt.axis('equal')  # 원의 형태를 유지 
plt.show()

In [ ]:
# 화성시 가구비율
labels = ['1인가구 비율', '2인가구 비율', '3인이상 가구비율']
sizes = [35.6, 19.6, 44.8]
colors = ['yellowgreen', 'gold', 'lightskyblue']
explode = (0, 0, 0.1)
plt.title('화성시 가구비율', size=30)
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
        autopct='%1.1f%%', shadow=True, startangle=90)
plt.axis('equal')  # 원의 형태를 유지 
plt.show()

##### 발표 내용에는 사용하지 않았지만 확인차 전통시장현화 데이터를 가져와 각 시에 있는 전통시장 갯수 확인

In [ ]:
local_market = pd.read_csv('datas/전통시장현황(개방표준).csv', encoding='CP949', low_memory=False)
local_market

###### 각 시/군에 있는 전통시장 갯수 확인 하는 코드

In [ ]:
place='수원시', '성남시', '고양시', '부천시', '용인시', '안산시', '안양시', '남양주시', '의정부시', '평택시', '시흥시', '화성시', '광명시', '파주시', '군포시', '광주시', '김포시', '이천시', '구리시', '양주시', '안성시', '포천시', '오산시', '하남시', '의왕시', '동두천시', '과천시', '여주군', '양평군', '가평군', '연천군'
count_market_list = []
for i in place:
    count_market = local_market['소재지도로명주소'].str.contains(i).sum()
    count_market_list.append(count_market)
count_market_list

####  경기도 내 31개의 시/군에 있는 시장 총 개수와 점포의 총 개수 

In [ ]:
df = pd.DataFrame(market_table)
local_market_insight=df.sort_values(['count_market','count_stores'], ascending=False)
local_market_insight['local_money']=local_market_insight.apply(lambda x: True if x.place=='안산시'or x.place=='시흥시'or x.place=='의왕시'or x.place=='안양시'or x.place=='성남시'or x.place=='과천시'or x.place=='포천시'or x.place=='평택시' else False, axis=1)
local_market_insight.reset_index(inplace=True, drop=True)
local_market_insight

### 4개 도시 확인(화성, 광주, 수원, 성남)

##### 데이터 불러오기

In [ ]:
age = pd.read_csv("datas/주민등록.csv", encoding = "CP949")
use_data = pd.read_csv("datas/payuse.csv", encoding = "CP949")

##### 인구 데이터

In [ ]:
age = age.iloc[:,:16] #사용할 컬럼

In [ ]:
age = age[age['연도'] >= 2019]
age = age[age['연도'] <= 2022]

In [ ]:
#시군명 컬럼 생성
age["시군명"] = age.행정구역명.str.split().str[1]

In [ ]:
age.dropna(subset = "시군명", inplace = True)
age

In [ ]:
peo = age[["시군명","총 인구수"]].groupby("시군명").mean() #시군 별 총 인구 수(평균)

In [ ]:
peo = peo/peo.sum() #비율

In [ ]:
peo = peo.loc[["성남시", "화성시" ,"수원시", "광주시"]] #4개 지역 총 인구 수 비율 데이터

##### 카드 사용 데이터

In [ ]:
use_data["기준년월"] = use_data["기준년월"].str.split('-').str[0] 

In [ ]:
use_data = use_data[["시군명","월별카드사용액(천원)"]].groupby("시군명").mean()/1000

In [ ]:
use_data = use_data/use_data.sum()
use_data

In [ ]:
use = use_data.loc[["성남시", "화성시" ,"수원시", "광주시"]]

In [ ]:
age_data = age[["시군명","총 인구수"]]

In [ ]:
age_data = age_data.groupby(["시군명"]).mean()

In [ ]:
age.시군명.unique()

In [ ]:
age_data = age_data.loc[["성남시", "화성시" ,"수원시", "광주시"]]

In [ ]:
age_data

#### 그래프

In [ ]:
import seaborn as sns

In [ ]:
sns.barplot(x = age_data.loc["성남시"].index, y = age_data.loc["성남시"].values)
plt.xticks(rotation = 90)
plt.title("성남시 인구 분포")
plt.show()

In [ ]:
sns.barplot(x = age_data.loc["화성시"].index, y = age_data.loc["화성시"].values)
plt.xticks(rotation = 90)
plt.title("화성시 인구 분포")
plt.show()

In [ ]:
sns.barplot(x = age_data.loc["광주시"].index, y = age_data.loc["광주시"].values)
plt.xticks(rotation = 90)
plt.title("광주시 인구 분포")
plt.show()

In [ ]:

plt.subplot(2, 2, 1)
plt.plot(age_data.loc["성남시"].index,age_data.loc["성남시"].values)
plt.xticks(rotation = 90)
plt.title("성남시 인구 분포")

plt.subplot(2, 2, 2)
plt.plot(age_data.loc["화성시"].index,age_data.loc["화성시"].values)
plt.xticks(rotation = 90)
plt.title("화성시 인구 분포")

plt.show()

In [ ]:
plt.plot(age_data.columns, age_data.loc['성남시'].values/age_data.loc['성남시'].values.sum(), label = "성남시")
plt.plot(age_data.columns, age_data.loc['화성시'].values/age_data.loc['화성시'].values.sum(), label = "화성시")
plt.plot(age_data.columns, age_data.loc['부천시'].values/age_data.loc['부천시'].values.sum(), label = "수원시")
plt.plot(age_data.columns, age_data.loc['광주시'].values/age_data.loc['광주시'].values.sum(), label = "광주시")
plt.xticks(rotation = 90)
plt.title("지역 별 인구 분포")
plt.legend()

plt.show()

In [ ]:
plt.bar(age_data.columns, age_data.loc['성남시'].values/age_data.loc['성남시'].values.sum(), alpha = 0.4, label = "성남시")
plt.bar(age_data.columns, age_data.loc['화성시'].values/age_data.loc['화성시'].values.sum(), alpha = 0.4, label = "화성시")
plt.xticks(rotation = 90)
plt.title("성남/화성 인구 분포")
plt.legend()
plt.show()

In [ ]:
sns.histplot(data = age_data.loc["광주시"], x = age_data.loc["광주시"].values, kde = True)
plt.xticks(labels = age_data.loc["광주시"].index)

In [ ]:
plt.plot(age_data.loc["광주시"].index,age_data.loc["광주시"].values)

In [ ]:
peo["월별카드사용액(천원)"] = use["월별카드사용액(천원)"]
peo.reset_index(inplace = True)

In [ ]:
pop_df2 = peo.set_index('시군명')
#.sort_values(by='인구평균', ascending=False)
# 인구 및 지역화폐 결제 금액에 대한 표
ax=pop_df2.sort_values(by='총 인구수', ascending=False).plot(kind='bar', figsize=(12, 8), legend=True, fontsize=12, color=['#E866BA','#98B3FF'])
ax.set_xlabel('지역명', fontsize=12)
ax.set_ylabel('비율', fontsize=12)
plt.title("인구 및 지역화폐 결제금액", fontsize=15)
ax.legend(['거주인구비율', '결제금액비율'], fontsize=12)
plt.show()